## Enter spotify **playlist URL**

In [ ]:
# Enter Playlist URL inside double quotes " "
playlist_url = ""
# Example: playlist_url = "https://open.spotify.com/playlist/21ff4YweZbISUbJWtfcfKQ?si=ef3121403c4a454d

### Click on Runtime and select **Run All**

In [ ]:
!pip install --upgrade google-api-python-client
!pip install git+https://github.com/ssuwani/pytube 
!pip install spotipy

In [ ]:
mkdir songs

In [ ]:
# importing libraries
import requests
import json
from googleapiclient.discovery import build
from pytube import YouTube
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [ ]:
# web scraping part instead of API
def youtube_Search(query):
  query = query.replace(" ", "+")
  url = "https://youtube-scrape.herokuapp.com/api/search?q="+query+"&page=1"

  page = requests.get(url)
  txt = page.text
  js = json.loads(txt)

  for i in range(len(js['results'])):
      vid_url = (js['results'][i]['video']['url'])
      break
  download(vid_url)
  print(vid_url)

In [ ]:
# pytube

def download(video_id):
  url = video_id
  print(url)
  yt = YouTube(url)
  # aud = yt.streams.filter(only_audio=True).all()
  aud = yt.streams.filter(only_audio = True).first()
   
  output = aud.download("/content/songs")
  base, ext = os.path.splitext(output)
  new = base + ".mp3"
  os.rename(output, new)

In [ ]:
client_id = "89bfb2d10c5549e7962657ef9d52e7fd"
client_secret = "878fee4f9baa424eafe05b298f5a69a1"

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id,
       client_secret=client_secret))

def sp_playlist_search():
  results = sp.playlist_items(playlist_url)
  albums=results['items']

  while results['next']:
    results=sp.next(results)

  albums.extend(results['items'])

  for album in albums:
    # print(album)
    artist_name = album['track']['artists'][0]['name']
    song_name = album['track']['name']
    print("\nDownloading ...")
    print(artist_name + " " + song_name)
    vid_name = artist_name + song_name

    try:
      youtube_Search(vid_name)
      print("Download finished.")
      time.sleep(1)
    except Exception as e:
      print("error: ", song_name, e)
      i = 0
      while i<3:
        try:
          youtube_Search(vid_name)
          print("Download finished.")
          break
        except:
          print("exception no: ", i)
          time.sleep(1)
          i+=1
          pass    

In [ ]:
sp_playlist_search()

#To download the folder

In [ ]:
!zip -r /content/songs.zip /content/songs

from google.colab import files
files.download("/content/songs.zip")